In [9]:
import pprint
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors

In [10]:
embedding_file = '/Volumes/ZHAO/CityLink/embedding/Tencent_AILab_ChineseEmbedding.txt'
wv_from_text = KeyedVectors.load_word2vec_format(embedding_file, binary=False)

In [12]:
wv_from_text.most_similar(positive=['女人', '皇帝'], negative=['男人'])

[('皇后', 0.8758628368377686),
 ('太后', 0.8436370491981506),
 ('皇太后', 0.8422078490257263),
 ('妃子', 0.8377931118011475),
 ('嫔妃', 0.8314578533172607),
 ('后宫嫔妃', 0.8293792009353638),
 ('后宫妃子', 0.8252508044242859),
 ('后妃', 0.8230723738670349),
 ('贵妃', 0.8157583475112915),
 ('妃嫔', 0.8150229454040527)]